# Building a research Agent with LangChain

In [1]:
# !pip install openai
# !pip install "langchain[all]"

In [1]:
from dotenv import load_dotenv

load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
# os.environ["OPENAI_API_KEY"] = "your-api-key"

In [2]:
from langchain_openai.chat_models import ChatOpenAI

In [3]:
llm_chat = ChatOpenAI(temperature=0)

In [4]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import tool

search = SerpAPIWrapper()

@tool
def web_search(query: str):
    """Searches the web given a user query and returns the most relevant links"""
    output = search.results(query)
    results = output["organic_results"]
    return [r["link"] for r in results]

web_search("Current resources on why pancakes are the best breakfast")

['https://www.quora.com/Why-do-pancakes-from-restaurants-taste-so-much-better-than-the-pancakes-I-make-at-home',
 'https://www.prnewswire.com/news-releases/packaged-facts-still-room-for-pancakes-and-waffles-at-americas-breakfast-table-300317750.html',
 'https://www.fitday.com/fitness-articles/nutrition/healthy-eating/healthy-breakfast-pancakes-vs-waffles.html',
 'https://www.reddit.com/r/Cooking/comments/wr3ngc/how_do_i_have_pancakes_for_breakfast_without/',
 'https://www.smithsonianmag.com/history/a-brief-history-of-pancakes-180981667/',
 'https://www.kingarthurbaking.com/blog/2021/10/26/best-pancake-tips',
 'https://www.snackandbakery.com/articles/94837-state-of-the-industry-2020-breakfast-products-combine-nutrition-and-good-taste',
 'https://www.turnipseedtravel.com/blog/the-best-pancakes-around-the-world',
 'https://shscourier.com/45785/commentary/which-breakfast-food-reigns-superior-pancakes-or-waffles/']

In [5]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")

prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [6]:
prompt.input_variables

['agent_scratchpad', 'input']

In [7]:
prompt.input_types

{'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]],
 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}

In [8]:
prompt.__dict__

{'name': None,
 'input_variables': ['agent_scratchpad', 'input'],
 'input_types': {'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]],
  'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]},
 'output_parser': None,
 'partial_variables': {},
 'metadata': None,
 'tags': None,
 'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
  MessagesPlaceholder(variable_name='chat_history', optional=True),
  HumanMessagePromptTe

To build an agent I need to do 2 essential things:

1. Connect a model to tools
2. Set up a loop in which the model continuously tries to solve the problem by either just calling the llm or calling a tool, observing the result and repeating until finished.

In [9]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [10]:
from langchain.agents import create_openai_tools_agent

tools = [web_search]

agent = create_openai_tools_agent(llm_chat, tools, prompt)

In [11]:
from langchain.agents import AgentExecutor


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
agent_executor.invoke({"input": "What are good resources to learn about learning new languages?",
                             "agent_scratchpad": []})



> Entering new AgentExecutor chain...

Invoking: `web_search` with `{'query': 'resources to learn new languages'}`


['https://nymag.com/strategist/article/how-to-learn-languages-at-home.html', 'https://www.theintrepidguide.com/top-language-learning-resources/', 'https://www.lingualift.com/blog/best-language-learning-resources/', 'https://www.reddit.com/r/languagelearning/comments/uhiuab/what_are_apps_or_websites_where_you_can_learn/', 'https://www.quora.com/What-resources-do-you-use-to-learn-languages', 'https://libguides.csudh.edu/modernlanguages/freeresources', 'https://www.alllanguageresources.com/', 'https://www.fluentu.com/blog/free-language-learning-websites/']I found some resources that can help you learn new languages:

1. [How to Learn Languages at Home - NYMag](https://nymag.com/strategist/article/how-to-learn-languages-at-home.html)
2. [Top Language Learning Resources - The Intrepid Guide](https://www.theintrepidguide.com/top-language-learning-resources/)
3. [Best Languag

{'input': 'What are good resources to learn about learning new languages?',
 'agent_scratchpad': [],
 'output': 'I found some resources that can help you learn new languages:\n\n1. [How to Learn Languages at Home - NYMag](https://nymag.com/strategist/article/how-to-learn-languages-at-home.html)\n2. [Top Language Learning Resources - The Intrepid Guide](https://www.theintrepidguide.com/top-language-learning-resources/)\n3. [Best Language Learning Resources - Lingualift](https://www.lingualift.com/blog/best-language-learning-resources/)\n4. [Apps or Websites to Learn Languages - Reddit](https://www.reddit.com/r/languagelearning/comments/uhiuab/what_are_apps_or_websites_where_you_can_learn/)\n5. [Resources for Learning Languages - Quora](https://www.quora.com/What-resources-do-you-use-to-learn-languages)\n6. [Free Language Learning Resources - CSUDH LibGuides](https://libguides.csudh.edu/modernlanguages/freeresources)\n7. [All Language Resources - All Language Resources](https://www.allla

In [13]:
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm_chat, tools, prompt)

react_agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


react_agent_executor.invoke({"input": "What is the best way to learn good programming practices?",
                             "agent_scratchpad": []})



> Entering new AgentExecutor chain...
It's important to gather information from reliable sources to learn about good programming practices.
Action: web_search
Action Input: "best way to learn good programming practices"['https://www.reddit.com/r/learnprogramming/comments/6bv3ux/how_do_you_practice_programming/', 'https://www.quora.com/How-can-I-learn-the-best-practices-in-programming', 'https://www.thinkful.com/blog/coding-best-practices/', 'https://www.educative.io/blog/coding-best-practices', 'https://www.datacamp.com/tutorial/coding-best-practices-and-guidelines', 'https://flatironschool.com/blog/best-websites-to-practice-coding-for-beginners/', 'https://codesigningstore.com/ultimate-programming-best-practices-guide', 'https://www.browserstack.com/guide/coding-standards-best-practices', 'https://medium.com/@josueparra2892/20-best-programming-practices-407df688b96e']There are many resources available to learn good programming practices.
Action: web_search
Action Input: "coding best

{'input': 'What is the best way to learn good programming practices?',
 'agent_scratchpad': [],
 'output': 'The best way to learn good programming practices is to read articles and guides dedicated to coding best practices.'}

Now let's build this same agent but using a more in depth approach going one level down in the abstraction layers.

Now, let's go down the latter of abstraction and build this same agent but leveraging the lower-level capabilities of LangChain's LCEL language. 

In [14]:
from langchain_openai.chat_models import ChatOpenAI

In [15]:
llm_chat = ChatOpenAI(model="gpt-3.5-turbo-16k",temperature=0)

In [16]:
from langchain.agents import tool
from langchain.utilities import SerpAPIWrapper

search = SerpAPIWrapper()

@tool
def web_search(query: str):
    """Searches the web given a user query and returns the most relevant links"""
    output = search.results(query)
    results = output["organic_results"]
    return [r["link"] for r in results]


tools = [web_search]

In [17]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a powerful research assistant.\
            Given a user query you will perform research on the \
                topic using the tools available."
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Bind tools to the LLM

In [19]:
from langchain_community.tools.convert_to_openai import format_tool_to_openai_tool

In [20]:
llm_with_tools = llm_chat.bind_tools(tools)

Now, all we have to do is import two last utility functions:
- a component for formatting intermediate steps (agent action, tool output pairs) to input messages that can be sent to the model, 
- and a component for converting the output message into an agent action/agent finish.

In [21]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

In [22]:
agent = (
    {
        "input": lambda x: x["input"], 
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [23]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [24]:
list(agent_executor.stream({"input": "What is the current best open source LLM available?"}))



> Entering new AgentExecutor chain...

Invoking: `web_search` with `{'query': 'best open source LLM'}`


['https://www.datacamp.com/blog/top-open-source-llms', 'https://github.com/Hannibal046/Awesome-LLM', 'https://www.reddit.com/r/LocalLLaMA/comments/17jmpn8/what_are_top_open_source_projects_in_llm_space/', 'https://www.width.ai/post/the-best-open-source-llms', 'https://blog.n8n.io/open-source-llm/', 'https://klu.ai/blog/open-source-llm-models', 'https://github.com/eugeneyan/open-llms', 'https://www.techopedia.com/6-best-open-source-llms-to-watch-out-for-in-2024', 'https://www.youtube.com/watch?v=L6nnNTVX-VI']There are several open source LLM (Language Model) projects available. Here are some resources that can help you find the best open source LLM:

1. [DataCamp Blog](https://www.datacamp.com/blog/top-open-source-llms): This blog post provides a list of top open source LLM projects.

2. [Awesome-LLM GitHub Repository](https://github.com/Hannibal046/Awesome-LLM): This GitHub reposi

[{'actions': [OpenAIToolAgentAction(tool='web_search', tool_input={'query': 'best open source LLM'}, log="\nInvoking: `web_search` with `{'query': 'best open source LLM'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_HiVnPSa0aULBRoUBQ8nBiVss', 'function': {'arguments': '{\n  "query": "best open source LLM"\n}', 'name': 'web_search'}, 'type': 'function'}]})], tool_call_id='call_HiVnPSa0aULBRoUBQ8nBiVss')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_HiVnPSa0aULBRoUBQ8nBiVss', 'function': {'arguments': '{\n  "query": "best open source LLM"\n}', 'name': 'web_search'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='web_search', tool_input={'query': 'best open source LLM'}, log="\nInvoking: `web_search` with `{'query': 'best open source LLM'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, '

Now, what if we want the agent to actually read the links and return a research report rather than just the urls?

In [25]:
import requests
from bs4 import BeautifulSoup


@tool
def scrape_text(url: str):
    """Scrapes the text from a given URL and returns it."""
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text
            return page_text[:10000]
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"

In [26]:
tools = [web_search, scrape_text]

In [27]:
llm_with_tools = llm_chat.bind_tools(tools)

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a powerful research assistant. Given a user query you will perform research on the topic using the tools available."
        ),
        ("user", "Scrape and summarize the contents from these urls: {input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [29]:
agent = (
    {
        "input": lambda x: x["input"], 
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [30]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [31]:
agent_executor.invoke({"input": "What are the current best practices to deploy LLM agents to production?"})



> Entering new AgentExecutor chain...

Invoking: `scrape_text` with `{'url': 'https://towardsdatascience.com/deploying-llm-agents-to-production-best-practices-7e4e8e8e8e8e'}`


Failed to retrieve the webpage: Status code 403
Invoking: `scrape_text` with `{'url': 'https://towardsdatascience.com/deploying-llm-agents-to-production-best-practices-7e4e8e8e8e8e'}`


Failed to retrieve the webpage: Status code 403
Invoking: `scrape_text` with `{'url': 'https://towardsdatascience.com/deploying-llm-agents-to-production-best-practices-7e4e8e8e8e8e'}`


Failed to retrieve the webpage: Status code 403
Invoking: `web_search` with `{'query': 'best practices to deploy LLM agents to production'}`


['https://www.qwak.com/post/best-practices-when-deploying-llms-in-production', 'https://www.linkedin.com/pulse/deploying-llms-production-anatomy-llm-applications-xenonstack', 'https://www.reddit.com/r/MachineLearning/comments/16fklqj/discussion_how_to_deploy_llm_in_production/', 'https://arize.com/blog-co

{'input': 'What are the current best practices to deploy LLM agents to production?',
 'output': 'Here are some resources that provide best practices for deploying LLM agents to production:\n\n1. [Best Practices for Deploying LLMs in Production](https://www.qwak.com/post/best-practices-when-deploying-llms-in-production): This article discusses the challenges and considerations for deploying LLMs in production and how Qwak helps streamline MLOps workflows.\n\n2. [Deploying LLMs in Production: The Anatomy of LLM Applications](https://www.linkedin.com/pulse/deploying-llms-production-anatomy-llm-applications-xenonstack): This LinkedIn article explores the deployment of LLMs in production and the components and reference architecture involved.\n\n3. [Discussion: How to deploy LLM in production](https://www.reddit.com/r/MachineLearning/comments/16fklqj/discussion_how_to_deploy_llm_in_production/): This Reddit thread discusses the challenges and approaches to deploying LLMs in production, with

We encounter a few issues when we try to just add the scrape function to the agent without any other consideration. 
How about we make this better by improving the inner chain of the agent?

In [33]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

input1 = "What are the current best practices to deploy LLM agents to production?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)



> Entering new AgentExecutor chain...

Invoking: `web_search` with `{'query': 'best practices to deploy LLM agents to production'}`


['https://www.qwak.com/post/best-practices-when-deploying-llms-in-production', 'https://www.linkedin.com/pulse/deploying-llms-production-anatomy-llm-applications-xenonstack', 'https://www.reddit.com/r/MachineLearning/comments/16fklqj/discussion_how_to_deploy_llm_in_production/', 'https://arize.com/blog-course/large-language-model-llm-deployment/', 'https://www.youtube.com/watch?v=NRjUMB9BrhQ', 'https://huyenchip.com/2023/04/11/llm-engineering.html', 'https://aws.amazon.com/blogs/machine-learning/learn-how-to-build-and-deploy-tool-using-llm-agents-using-aws-sagemaker-jumpstart-foundation-models/', 'https://www.seldon.io/deploying-large-language-models-in-production']Here are some current best practices for deploying LLM agents to production:

1. Understand the requirements: Before deploying an LLM agent, it is important to understand the specific requir

In [34]:
result

{'input': 'What are the current best practices to deploy LLM agents to production?',
 'chat_history': [HumanMessage(content='What are the current best practices to deploy LLM agents to production?'),
  AIMessage(content='Here are some current best practices for deploying LLM agents to production:\n\n1. Understand the requirements: Before deploying an LLM agent, it is important to understand the specific requirements of your application. This includes considering factors such as the size of the model, the computational resources needed, and the expected workload.\n\n2. Model selection and fine-tuning: Choose the right LLM model for your application. Consider factors such as model size, performance, and compatibility with your infrastructure. Fine-tune the model on your specific dataset to improve its accuracy and performance.\n\n3. Infrastructure considerations: Ensure that your infrastructure can handle the computational requirements of the LLM agent. This may involve using high-perfor